In [1]:
#install hugging face transformers and torch
!pip install transformers
!pip install torch

You should consider upgrading via the '/Users/admin/.pyenv/versions/3.7.7/envs/finbert/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/admin/.pyenv/versions/3.7.7/envs/finbert/bin/python -m pip install --upgrade pip' command.


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import numpy as np

#import pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

#import pretrained model
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [2]:
#import csv and sample => CHANGE PATH
headlines_df = pd.read_csv('../data/analyst_ratings_processed.csv')
headlines_df = headlines_df.sample(10, random_state=42)

In [3]:
#transform to numpy - shuffle and put only headlines in list
headlines_array = np.array(headlines_df)
np.random.shuffle(headlines_array)
headlines_list = list(headlines_array[:,1])

headlines_list[:5]

['The Deal Says Citrix Is Not Exploring A Possible Sale',
 'ONEOK Partners to Participate in Bakken Day',
 'Shares of Broadsoft to Resume Trade at 4:35PM EST',
 'Asterias Provides 6 Mo. Data Readout From Its AST-OPC1 Phase 1/2a Clinical Trial, Says Results Continue to Demonstrate Favorable Safety Profile, Potential for Durable Cell Engraftment at the Injury Site and Improved Motor Function',
 'eFuture Announces Appointment of Ms. Ping Yu As CFO']

In [4]:
#tokenize inputs
inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')


In [5]:
#generate output model
outputs = model(**inputs)
print(outputs.logits)

tensor([[-0.3778, -1.8577,  2.5023],
        [ 0.4874, -2.4585,  2.1729],
        [-0.6469, -1.5176,  2.5074],
        [ 2.1141, -2.0579, -1.3233],
        [-0.7994, -1.0177,  2.0862],
        [ 0.2181,  1.3837, -1.6399],
        [ 2.0557, -1.4729, -1.7830],
        [-0.2203, -1.6770,  2.2643],
        [ 0.4043, -2.4181,  2.0182],
        [-0.2195, -1.7844,  2.4427]], grad_fn=<AddmmBackward>)


In [6]:
#translate outputs to predictions via softmax
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.0525, 0.0120, 0.9355],
        [0.1551, 0.0082, 0.8368],
        [0.0402, 0.0168, 0.9429],
        [0.9546, 0.0147, 0.0307],
        [0.0507, 0.0408, 0.9085],
        [0.2292, 0.7351, 0.0357],
        [0.9516, 0.0279, 0.0205],
        [0.0756, 0.0176, 0.9068],
        [0.1644, 0.0098, 0.8258],
        [0.0644, 0.0135, 0.9222]], grad_fn=<SoftmaxBackward>)


In [7]:
#predictions to df
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

In [8]:
#sorted df
df.sort_values(by='Positive', ascending=False)

,Headline,Positive,Negative,Neutral
3,Asterias Provides 6 Mo. Data Readout From Its ...,0.954589,0.014721,0.030690
6,"Courier Corp Holder Gamco Reports 6.97%, Up Fr...",0.951596,0.027925,0.020479
5,UPDATE: Transocean and Chevron Moving off Lows...,0.229157,0.735099,0.035745
8,Top Stocks In The Surety & Title Insurance Ind...,0.164415,0.009777,0.825807
1,ONEOK Partners to Participate in Bakken Day,0.155090,0.008151,0.836759
7,Cedar Shopping Centers Closes Purchase of New ...,0.075591,0.017613,0.906796
9,Marsh Subsidiary Marsh & McLennan To Acquire B...,0.064367,0.013459,0.922174
0,The Deal Says Citrix Is Not Exploring A Possib...,0.052512,0.011955,0.935533
4,eFuture Announces Appointment of Ms. Ping Yu A...,0.050713,0.040770,0.908517
2,Shares of Broadsoft to Resume Trade at 4:35PM EST,0.040232,0.016843,0.942925


In [9]:
path = '/Users/admin/code/Michiel-DK/finbert/notebooks/models'

In [10]:
#save weights model
torch.save(model.state_dict(), f"{path}pytorch_model.bin")

In [11]:
#load weights model
model = AutoModelForSequenceClassification.from_pretrained(f"{path}", local_files_only=True, num_labels=3)

In [16]:
#save weights model
torch.save(tokenizer, f"{path}tokenizer.bin")

In [20]:
tokenizer = AutoTokenizer.from_pretrained(f"{path}", use_fast=True)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType